In [ ]:
import tempfile, shutil, os

In [ ]:
slack_url = os.environ.get('SLACK_API_URL', None)
namespace =  'rcos'
pod_lines =  !kubectl get pod -l app=binder -l name=binder -o jsonpath="{.items[0].metadata.name}"
pod = pod_lines[0]
log_dir = '~/.jenkins/binder-logs' # 必ず永続化されているディレクトリを設定するようにする
(slack_url, namespace, pod, log_dir)

In [ ]:
from dateutil import tz
from datetime import datetime, timedelta,timezone

In [ ]:
!mkdir -p {log_dir}

In [ ]:
# ログディレクトリの中で、最も最新の日付のログを収集のスタートとする
#（kubectl logsのは開始時刻が指定できるが、終了時刻が指定できない）
import os
import dateutil.parser
logs = !ls {log_dir}
max_d = datetime.min
max_d = max_d.replace(tzinfo=tz.UTC)
for log in logs:
    name_without_ext = os.path.splitext(os.path.basename(log))[0]
    try:
        d = dateutil.parser.isoparse(name_without_ext)
        d = d.replace(tzinfo=tz.UTC)
        print(d)
        if max_d < d:
            max_d = d
    except Exception as e:
        print(e)
    
if max_d != datetime.min:
    start_timestamp = max_d.isoformat()
else:
    start_timestamp = None
start_timestamp

In [ ]:
now = datetime.now(tz.UTC).isoformat()
if start_timestamp:
    !kubectl logs {pod} -n {namespace} --since-time={start_timestamp} > {log_dir}/{now}.log
else :
    !kubectl logs {pod} -n {namespace} > {log_dir}/{now}.log
#     requests.post(slack_url, json=data)